In [16]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [17]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [18]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [19]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-torch',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 60 commands: 
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_temporal_1/0_0_0.0"  --dtype="float32" --device="cuda:0" --img_size=64 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.1 --prompts="a single biological cell,two biological cells"                                                                            --spatial_scales="1." --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_temporal_softmax=0.0  --rollout_steps=256 --bptt_steps=16 --pool_size=256 --bs=8 --lr=0.0003 --n_iters=100000 --clip_grad_norm=1.0
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_temporal_1/0_0_0.01" --dtype="float32" --device="cuda:0" --img_size=64 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.1 --prompts="a 

In [20]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42


In [21]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [22]:
print('\n'.join(txt.split('\n')[:24]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-torch
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42/job_00003.status
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_temporal_1/0_0_0.1"  --dtype="float32" --device="cuda:0" --img_size=64 --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.1 --prompts="a single biological cell,two biological cells"                                                                            --spatial_scales="1." --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_temporal_softmax=0.1  --rollout_steps=256 --bptt_steps=16 --pool_size=256 --bs=8 --lr=0.0003 --n_iters=100000 --clip_grad_norm=1.0 &> /home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2

In [23]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_08_03_08_19_42/gpu_7.sh


# Track Experiment

In [22]:
import subprocess
a = subprocess.run(['ps', 'ux'], stdout=subprocess.PIPE).stdout.decode('utf-8')

In [25]:
[l for l in a.split('\n') if "bash" in l and "gpu" in l]

['akarshk+ 1271851  0.0  0.0   2480   572 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_4.sh',
 'akarshk+ 1271852  0.0  0.0   2480   516 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_5.sh',
 'akarshk+ 1271853  0.0  0.0 219972  3408 ?        S    04:58   0:00 bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_4.sh',
 'akarshk+ 1271854  0.0  0.0   2480   512 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_6.sh',
 'akarshk+ 1271855  0.0  0.0 219972  3200 ?        S    04:58   0:00 bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_5.sh',
 'akarshk+ 1271856  0.0  0.0   2480   516 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_7.sh',
 'akarshk+ 1271857  0.0  0.0 219972  3460 ?        S    04:58   0:00 bash 

Terminated
Terminated
Terminated
Terminated


In [64]:
from collections import defaultdict

statuses = []
for job_id in range(len(commands)):
    try:
        with open(f"{log_dir}/job_{job_id:05d}.status", "r") as f:
            status = int(f.read().strip())
            statuses.append(status)
    except Exception as e:
        pass
        # print(e)
        # break
statuses = np.array(statuses)

In [65]:
(statuses==0).sum(), len(commands)

(0, 60)

In [66]:
np.unique(statuses, return_counts=True)

(array([], dtype=float64), array([], dtype=int64))

In [31]:
for job_id in list(range(len(commands))):
    print()
    print(f'------------------------------------------------------ {job_id} ------------------------------------------------------')
    print()
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        # pass
        break
        print(e)


------------------------------------------------------ 0 ------------------------------------------------------

1%|          | 806/100000 [00:27<1:01:46, 26.76it/s, loss=-0.292]

------------------------------------------------------ 1 ------------------------------------------------------

1%|          | 776/100000 [00:27<1:04:06, 25.79it/s, loss=-0.282]

------------------------------------------------------ 2 ------------------------------------------------------

1%|          | 928/100000 [00:32<1:04:17, 25.68it/s, loss=-0.275]

------------------------------------------------------ 3 ------------------------------------------------------

1%|          | 894/100000 [00:32<1:04:04, 25.78it/s, loss=-0.229]

------------------------------------------------------ 4 ------------------------------------------------------



In [14]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_08_02_05_16_15'